In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from astropy.table import Table
%matplotlib inline

In [ ]:
data_path = "../surveysim/output/"
plan_files = glob.glob(os.path.join(data_path, "plan_*"))
progress_files = glob.glob(os.path.join(data_path, "progress_*"))
progress_files = sorted(progress_files)
plan_files = sorted(plan_files)

In [ ]:
a = Table.read(plan_files[-1])
b = Table.read(progress_files[-1])
print(a[:10])
print(b[:10])
print(a.keys())
print(b.keys())

In [ ]:
print(list(set(a['group'])))
list(set(b['status']))

In [ ]:
tiling_file = os.path.join(os.environ['DESIMODEL'], 'data/footprint/', 'desi-tiles.fits')
tiling_data = Table.read(tiling_file)
tiling_data.keys()

#get the program of the unobserved tiles
program = tiling_data['PROGRAM'][np.in1d(tiling_data['TILEID'], b['tileid'][b['status']==0])]
n_all = len(b)
print(n_all)
print('indesi', np.count_nonzero(tiling_data['IN_DESI']!=0))
print(np.count_nonzero(program=='DARK')/n_all, 
      np.count_nonzero(program=='BRIGHT')/n_all, 
      np.count_nonzero(program=='GRAY')/n_all)

In [ ]:
n_progress = len(progress_files)
n_plan = n_progress - 1
n_0 = np.zeros(n_progress)
n_2 = np.zeros(n_progress)
for i in range(n_plan):
    a = Table.read(progress_files[i+1])
    n_0[i+1] = np.count_nonzero(a['status']==0)
    n_2[i+1] = np.count_nonzero(a['status']==2)
    print(i, n_0[i+1])
#    plt.scatter(a['ra'][a['status']==2], a['dec'][a['status']==2], alpha=0.1)

In [ ]:
plt.plot(n_0)
plt.plot(n_2)